In [13]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import string

# Processing the grades data

In [14]:
grades_data = pd.read_csv('data/Taylor_Swift_Songs_Data/Grades.csv', header=None)

In [15]:
# Working on the grades

grades = grades_data.iloc[1:, :12].reset_index(drop=True)
grades.columns = grades.iloc[0]
grades = grades.drop(0).reset_index(drop=True)

grades.columns = ['song_name', 'album', 'happy_sad', 'relationship', 'feelings_of_self', 'glass_half_full', 'stages', 'tempo', 'seriousness', 'future_prospects', 'feelings_of_male', 'togetherness']
grades['song_name'] = grades['song_name'].apply(lambda x: str(x).lower())
grades.head()

,song_name,album,happy_sad,relationship,feelings_of_self,glass_half_full,stages,tempo,seriousness,future_prospects,feelings_of_male,togetherness
0,cold as you,Taylor Swift,-10,-8,-1,-3,-3,-3,-3,-3,-1,-1
1,i'm only me when i'm with you,Taylor Swift,9,10,3,3,1,2,2,2,3,3
2,invisible,Taylor Swift,-1,-4,0,-2,1,0,0,0,-1,-3
3,mary's song,Taylor Swift,5,12,0,2,1,2,3,3,3,3
4,our song,Taylor Swift,5,6,2,2,1,0,1,1,3,1


# Processing the lyrics data (Version 1, not usable)

In [16]:
album1 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/01-taylor_swift.csv')
album2 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/02-fearless_taylors_version.csv')
album3 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/03-speak_now_deluxe_package.csv')
album4 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/04-red_deluxe_edition.csv')
album5 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/05-1989_deluxe.csv')
album6 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/06-reputation.csv')
album7 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/07-lover.csv')
album8 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/08-folklore_deluxe_version.csv')
album9 = pd.read_csv('data/Taylor_Swift_Songs_Data/lyrics/09-evermore_deluxe_version.csv')

In [17]:
lyrics = pd.concat([album1, album2, album3, album4, album5, album6, album7, album8, album9], axis=0).reset_index(drop=True)
lyrics['track_title'] = lyrics['track_title'].apply(lambda x: str(x).lower())
lyrics['track_title'] = [x.split('(taylor’s version)')[0] for x in lyrics['track_title']]
lyrics['track_title'] = [x.rstrip() for x in lyrics['track_title']]
lyrics['track_title'] = [''.join(x.split('\u200b')).rstrip() for x in lyrics['track_title']]
lyrics['track_title'] = ["'".join(x.split('’')) for x in lyrics['track_title']]

# Processing lyrics data (Version 2)

In [18]:
def clean_text(text):
    
    # Some light data cleaning - you will need to adjust based on your data
    text = text.replace('See Taylor Swift LiveGet tickets as low as $270', ' ') # remove ad
    text = text.replace('See Taylor Swift LiveGet tickets as low as $373', ' ') # remove ad
    text = text.replace('You might also like', ' ') # remove ad
    text = re.sub('\d*Embed', ' ', text) # remove ending text with number + Embed
    
    return text

In [19]:
# Specify the folder names with the lyric data from Genius
directory_paths = ['data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Taylor-Swift/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Fearless/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Speak-Now/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Red/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_1989/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Reputation/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Lover/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_folklore/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_evermore/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_Midnights/',
                   'data/Taylor_Swift_Songs_Data/Taylor_Swift_Genius/Taylor-Swift_NA/']

In [20]:
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', 0)

ts_lyrics = pd.DataFrame({"album": [],
                          "song_name": [],
                          "lyrics": []})

idx = 0

for i, album in enumerate(directory_paths):
    
    album_name = album.split('/')[-2].split('_')[-1].replace("-", " ")
        
    for song in Path(album).glob('*.txt'):

        song_name = str(song).replace("-", " ").split("\\")[-1][:-4]
        song_name = song_name.split('[')[0].strip()
        song_name = ''.join(song_name.split('\u200b'))
        song_name = song_name.split("(Taylor's Version)")[0].rstrip()
        song_name = song_name.split("(Oh My My My)")[0].rstrip()
        song_name = song_name.replace('’', "'")
        song_name = song_name.replace('questionMark', '?')
                
        full_text = open(song, encoding="utf-8")
        lyrics_list = full_text.readlines()[1:] #read()
        lyrics = ' '.join(lyrics_list)
        lyrics = clean_text(lyrics)
        full_text.close()
        
        ts_lyrics.loc[idx] = [album_name, song_name, lyrics]
        idx += 1

In [21]:
ts_lyrics['song_name'] = ts_lyrics['song_name'].apply(lambda x: str(x).lower())
set(grades['song_name']) - set(ts_lyrics['song_name'])

set()

In [22]:
rag_dataset = grades.merge(ts_lyrics, on='song_name', how='left').reset_index(drop=True)
rag_dataset = rag_dataset.drop('album_y', axis=1)
rag_dataset = rag_dataset.rename({'album_x':'album'}, axis=1)
rag_dataset = rag_dataset.drop(25, axis=0).reset_index(drop=True)  # Drop duplicated song
rag_dataset['lyrics'] = rag_dataset['lyrics'].apply(lambda x: [verse for verse in x.split('\n') if verse != '' and verse != ' ' and verse != '   '])
rag_dataset.head()

,song_name,album,happy_sad,relationship,feelings_of_self,glass_half_full,stages,tempo,seriousness,future_prospects,feelings_of_male,togetherness,lyrics
0,cold as you,Taylor Swift,-10,-8,-1,-3,-3,-3,-3,-3,-1,-1,"[You have a way of coming easily to me, And when you take, you take the very best of me, So I start a fight 'cause I need to feel something, And you do what you want 'cause I'm not what you wanted, Oh, what a shame, what a rainy ending, Given to a perfect day, Just walk away, ain't no use defending, Words that you will never say, And now that I'm sitting here thinking it through, I've never been anywhere cold as you, You put up walls and paint them all a shade of gray, And I stood there loving you and wished them all away, And you come away with a great little story, Of a mess of a dreamer with the nerve to adore you, Oh, what a shame, what a rainy ending, Given to a perfect day, So just walk away, ain't no use defending, Words that you will never say, And now that I'm sitting here thinking it through, I've never been anywhere cold as you, You never did give a damn thing, honey, But I cried, cried for you, And I know you wouldn't have told nobody, If I died, died for you, died for you, Oh, what a shame, what a rainy ending, Given to a perfect day, oh, Every smile you fake is so condescending, Counting all the scars you made, And now that I'm sitting here thinking it through, I've never been anywhere cold as you, Ooh-oh ]"
1,i'm only me when i'm with you,Taylor Swift,9,10,3,3,1,2,2,2,3,3,"[Friday night beneath the stars, In a field behind your yard, You and I are painting pictures in the sky, And sometimes, we don't say a thing, Just listen to the crickets sing, Everything I need is right here by my side, And I know everything about you, I don't wanna live without you, I'm only up when you're not down, Don't wanna fly if you're still on the ground, It's like no matter what I do, Well, you drive me crazy half the time, The other half, I'm only tryin' to, Let you know that what I feel is true, And I'm only me when I'm with you, Just a small-town boy and girl, Livin' in a crazy world, Tryna figure out what is and isn't true, And I don't try to hide my tears, My secrets or my deepest fears, Through it all, nobody gets me like you do, And you know everything about me, You say that you can't live without me, I'm only up when you're not down, Don't wanna fly if you're still on the ground, It's like no matter what I do, Well, you drive me crazy half the time, The other half, I'm only tryin' to, Let you know that what I feel is true, And I'm only me when I'm with you, When I'm with anybody else, It's so hard to be myself, And only you can tell, That I'm only up when you're not down, Don't wanna fly if you're still on the ground, It's like no matter what I do, Well, you drive me crazy half the time, The other half, I'm only tryin' to, Let you know that what I feel is true, And I'm only me, Who I wanna be, Well, I'm only me when I'm with you, With you, Uh-huh-huh, yeah ]"
2,invisible,Taylor Swift,-1,-4,0,-2,1,0,0,0,-1,-3,"[She can't see the way your eyes, Light up when you smile, She'll never notice how you stop and stare, Whenever she walks by, And you can't see me wanting you the way you want her, But you are everything to me, And I just wanna show you, She don't even know you, She's never gonna love you like I want to, And you just see right through me, But if you only knew me, We could be a beautiful miracle, unbelievable, Instead of just invisible, Oh, yeah, oh, There's a fire inside of you, That can't help but shine through, She's never gonna see the light, No matter what you do, And all I think about is how to make you think of me, And everything that we could be, And I just wanna show you, She don't even know you, She's never gonna love you like I want to, And you just see right through me, But if you only knew me, We could be a beautiful miracle, unbelievable, Instead of just invisible, Like shadows in a faded light, Oh, w

In [23]:
rag_dataset.to_csv('data/cleaned_data/rag_dataset.csv', index=False)